In [1]:
# Cell 1

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("Titanic-Dataset.csv")
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
# Cell 2

df = df.dropna()

target = "Survived"

# Encode categorical columns
for col in df.columns:
    if df[col].dtype == "object":
        df[col], _ = pd.factorize(df[col])

# Features and target
X = df.drop(columns=[target]).values
y = df[target].values

# Normalize numeric values
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 80:20 split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [3]:
# Cell 3

def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2))


In [4]:
# Cell 4

class KNNClassifier:
    def __init__(self, k):
        self.k = k
    
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
    
    def predict_one(self, x):
        # Compute distances
        distances = np.array([euclidean_distance(x, x_train) for x_train in self.X_train])
        
        # Get k nearest indices
        k_idx = distances.argsort()[:self.k]
        
        # Get the majority class
        k_labels = self.y_train[k_idx]
        values, counts = np.unique(k_labels, return_counts=True)
        return values[np.argmax(counts)]
    
    def predict(self, X):
        return np.array([self.predict_one(x) for x in X])


In [5]:
# Cell 5

def macro_f1(y_true, y_pred):
    classes = np.unique(y_true)
    f1_scores = []

    for c in classes:
        tp = np.sum((y_pred == c) & (y_true == c))
        fp = np.sum((y_pred == c) & (y_true != c))
        fn = np.sum((y_pred != c) & (y_true == c))

        precision = tp / (tp + fp + 1e-10)
        recall = tp / (tp + fn + 1e-10)
        f1 = 2 * precision * recall / (precision + recall + 1e-10)

        f1_scores.append(f1)

    return np.mean(f1_scores)


In [6]:
# Cell 6

results = {}

for k in [3, 5, 7]:
    model = KNNClassifier(k=k)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    f1 = macro_f1(y_test, y_pred)
    results[k] = f1
    print(f"K = {k}, Macro F1 Score = {f1:.4f}")


K = 3, Macro F1 Score = 0.5899
K = 5, Macro F1 Score = 0.7376
K = 7, Macro F1 Score = 0.6300


In [7]:
# Cell 7

print("\nFinal Comparison (Macro F1 Scores):")
for k, score in results.items():
    print(f"K = {k}:  {score:.4f}")



Final Comparison (Macro F1 Scores):
K = 3:  0.5899
K = 5:  0.7376
K = 7:  0.6300


In [8]:
# Cell 8

def accuracy(y_true, y_pred):
    return np.sum(y_true == y_pred) / len(y_true)

print("\nAccuracy Scores:")
for k in results.keys():
    model = KNNClassifier(k=k)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy(y_test, y_pred)
    print(f"K = {k}, Accuracy = {acc:.4f}")



Accuracy Scores:
K = 3, Accuracy = 0.6486
K = 5, Accuracy = 0.7568
K = 7, Accuracy = 0.6757
